# Diabetes Dataset Preparation Notebook

This notebook serves as the first step in a series of notebooks aimed at analyzing a diabetes dataset. In this notebook, we'll focus on data acquisition, sampling, and preparation for model training in subsequent steps. The key processes include:

1. **Environment Setup**:
    - Importing necessary libraries.
    - Establishing Azure ML client connection.

2. **Data Acquisition**:
    - Defining the data paths.
    - Loading data into `MLTable` objects using `mltable.from_delimited_files()`.

3. **Data Sampling**:
    - Random sampling of data using `mltable` functionality.
    - Converting sampled data to Pandas DataFrame for further analysis.

4. **Data Saving and Versioning**:
    - Saving the data to disk in MLTable format.
    - Creating or updating the data asset in Azure ML with versioning.

5. **Data Splitting**:
    - Splitting the data into training and testing sets using `train_test_split`.
    - Exploring the split data (optional).

6. **Data Persistence**:
    - Saving the split data to disk for future use.

This notebook sets the stage for the following notebooks in this series:
- **Train and Deploy a Model**: Building, training, and deploying a machine learning model.
- **Invoke a Real-Time Endpoint**: Making real-time predictions using the deployed model.
- **Create Synthetic Data**: Generating synthetic data for further analysis.
- **Explore Collected Data from Production**: Analyzing data collected from the production environment to gain insights and improve the model.

**Note**: This notebook assumes familiarity with Azure ML, Scikit-learn, and the `mltable` library for tabular data manipulation and analysis. For more detailed information on `mltable`, please refer to the [official documentation](link_to_mltable_documentation).


In [ ]:
# !pip install mltable scikit-learn

In [ ]:
# Import necessary libraries
import mltable  # Library for working with tabular data
import os  # Operating system interfaces
import time  # Time access and conversions
from azure.ai.ml import MLClient  # Azure ML Client for managing ML assets
from azure.ai.ml.entities import Data  # Data entity class for Azure ML
from azure.ai.ml.constants import AssetTypes  # Constants for Azure ML asset types
from azure.identity import DefaultAzureCredential  # Credential class for Azure authentication

## Load Data

In this section, we'll load the diabetes dataset from two distinct files using `mltable.from_delimited_files()`. The paths variable holds dictionaries that specify the URLs of these CSV files. Allowed keys are `file`, `folder`, and `pattern`.

In [ ]:
# Define the file paths
paths = [
    {
        "pattern": "https://raw.githubusercontent.com/MicrosoftLearning/mslearn-dp100/main/data/diabetes.csv"
    },
    {
        "file": "https://raw.githubusercontent.com/MicrosoftLearning/mslearn-dp100/main/data/diabetes2.csv"
    },
    # {
    #     "folder": "https://raw.githubusercontent.com/MicrosoftLearning/mslearn-dp100/main/data/"
    # }
]

# Create an MLTable object
tbl = mltable.from_delimited_files(paths)

tbl.show(5)


The `mltable.from_delimited_files()` function simplifies the loading process by auto-detecting the delimiter and establishing appropriate column names and data types, encapsulating the data within an MLTable object for easy manipulation and analysis.

## Sample Data
For initial exploration and model testing, we'll draw a random sample from the loaded data. The `tbl.take_random_sample()` method facilitates this, where `probability` sets the row selection chance, and `seed` ensures reproducibility.

In [ ]:
# Take a random sample of the table
tbl_sample = tbl.take_random_sample(probability=0.001, seed=735)

# Convert to Pandas DataFrame
df = tbl_sample.to_pandas_dataframe()

# Display the first few rows
df.head()

In [ ]:
median_patient_id = str(int(df["PatientID"].median()))
median_patient_id

In [ ]:
# tbl_sample.random_split(percent=.5)
tbl_dev = tbl.filter(f'PatientID <= {median_patient_id}')
tbl_prod = tbl.filter(f'PatientID > {median_patient_id}')

## Enriching Production Data with Dates

In [ ]:
tbl_prod_features, tbl_prod_labels = tbl_prod.drop_columns(["Diabetic", "PatientID"]), tbl_prod.keep_columns(["Diabetic"])
tbl_prod_features.show(5)

In [ ]:
import numpy as np
from datetime import datetime, timedelta

# Assuming df is your DataFrame
df_prod_features = tbl_prod_features.to_pandas_dataframe()

# Get today's date
today = datetime.today()

# Generate random number of days between -30 and 30
random_days = np.random.randint(-30, 30, size=len(df_prod_features))

# Create date column with random dates
df_prod_features['date'] = [(today + timedelta(days=int(d))).isoformat() + 'Z' for d in random_days]

df_prod_features.head(5)


## Data Saving and Versioning

Post sampling, we persist the `MLTable` object to disk, and define a version for the data asset based on the current UTC time. This versioning can be invaluable for traceability and managing data versions in machine learning workflows.

In [ ]:
# Set the version number of the data asset to the current UTC time
VERSION = time.strftime("%Y.%m.%d.%H%M%S", time.gmtime())

In [ ]:
# delete model directory if it exists
import shutil
if os.path.exists("data"):
    shutil.rmtree("data")

os.makedirs("data")
os.makedirs("data/train")
os.makedirs("data/test")
os.makedirs("data/eval")
os.makedirs("data/prod")
os.makedirs("data/prod/features")
os.makedirs("data/prod/labels")

In [ ]:

# Save the table to disk in MLTable format
tbl_prod_labels.save("./data/prod/labels")
tbl_dev.save("./data/dev")

print(os.listdir("./data/prod/features"))

with open("./data/prod/labels/MLTable") as f:
    print(f.read())


Connect to the AzureML workspace and define the data asset. The `create_or_update` method is employed to either create a new data asset or update an existing one in AzureML.

In [ ]:
df_prod_features.to_parquet("./data/prod/features/data.parquet")

# Connect to the AzureML workspace
ml_client = MLClient.from_config(credential=DefaultAzureCredential())

# Define the data asset
my_data = Data(
    path="./data/prod/features",
    type=AssetTypes.URI_FOLDER,
    description="Example data from the diabetes dataset (Production)",
    name="diabetes-urifolder-production",
    version=VERSION,
)

# Create or update the data asset in AzureML
ml_client.data.create_or_update(my_data)


In [ ]:


# Connect to the AzureML workspace
ml_client = MLClient.from_config(credential=DefaultAzureCredential())

# Define the data asset
my_data = Data(
    path="./data/prod/labels",
    type=AssetTypes.MLTABLE,
    description="Example data from the diabetes dataset (Production Labels)",
    name="diabetes-mltable-production-labels",
    version=VERSION,
)

# Create or update the data asset in AzureML
ml_client.data.create_or_update(my_data)


# Connect to the AzureML workspace
ml_client = MLClient.from_config(credential=DefaultAzureCredential())

# Define the data asset
my_data = Data(
    path="./data/dev/",
    type=AssetTypes.MLTABLE,
    description="Example data from the diabetes dataset (Train, Test, Eval)",
    name="diabetes-mltable-dev",
    version=VERSION,
)

# Create or update the data asset in AzureML
ml_client.data.create_or_update(my_data)

## Split Data
The traditional approach would be to utilize `train_test_split` to segregate the data into training and testing sets, facilitating model validation on unseen data. 

While the above approach works well, we'll use the native MLTable functionality, which makes it quite easy to split data into train, test, and validation.

In [ ]:
tbl_train, tbl_test_eval = tbl_dev.random_split(percent=.7)
tbl_test, tbl_eval = tbl_test_eval.random_split(percent=.5)


Once we use sample, we need to store the file locally, create a new MLTable pointing to that file, and register the Data Asset.

In [ ]:
tables = {"train": tbl_train, "test": tbl_test, "eval": tbl_eval}

for stage in tables.keys():
    print(f"Registering stage: {stage}")
    path = f"./data/{stage}/"
    parquet_path = path + f"{stage}.parquet" 
    print(parquet_path)

    table = tables.get(stage)

    df = table.to_pandas_dataframe()

    # Save files locally
    df.to_parquet(parquet_path)

    # Create new MLTable file, pointing towards the local files.
    tbl = mltable.from_parquet_files(paths=[{"file": parquet_path}])
    tbl.save(path)
    
    # Define Data Asset
    my_data = Data(
        path=path,
        type=AssetTypes.MLTABLE,
        description=f"Example data from the diabetes dataset ({stage})",
        name=f"diabetes-mltable-{stage}",
        version=VERSION,
    )

    # Create or update the data asset in AzureML
    # try:
    ml_client.data.create_or_update(my_data)
    # except TypeError as e:
    #     print(e)

This sectioning enables a streamlined workflow from data loading to splitting, ensuring the data is readily accessible for subsequent analysis and modeling in following notebooks.

## Next step

We've created some data assets, that lay the foundation of this accelerator. Proceed with the next notebook to understand how to train and deploy Models in AzureML.